# 시험장 환경 정보

Python: 3.7.4 (tags/v3.7.4:e09359112e, Jul  8 2019, 20:34:20) [MSC v.1916 64 bit (AMD64)]

|모듈|버젼|
|----|----|
|pandas|0.25.1|
|numpy|1.18.5|
|sklearn|0.21.3|
|scipy|1.5.2|
|mlxtend|0.15.0.0|
|statsmodels|0.11.1|
|xgboost|0.8|

**강사: 멀티캠퍼스 강선구(sunku0316.kang@multicampus.com, sun9sun9@gmail.com)**

In [1]:
# 실행 환경 확인

import pandas as pd
import numpy as np
import sklearn
import scipy
import statsmodels
import mlxtend
import sys
import xgboost as xgb

print(sys.version)
for i in [pd, np, sklearn, scipy, mlxtend, statsmodels, xgb]:
    print(i.__name__, i.__version__)

ModuleNotFoundError: No module named 'xgboost'

In [2]:
# 시각화 모듈 설정
# 참고용 차트를 출력하기 위함

import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib as mpl
import matplotlib.font_manager as fm
plt.rc('font', family='Malgun Gothic')
mpl.rcParams['axes.unicode_minus'] = False

# 문제 개요

다음은 폴더블 폰의 힌지에 들어가는 스프링 내구력을 테스트한 실험 결과이다. 

스프링 측정값과 스프링에 가한 부하 정보와 함께, 테스트 통과/실패 (failure) 결과가 기재되어 있다. 

개발부서는 테스트 비용을 줄이기 위해 failure 여부를 맞추는 모델을 만들고자 한다.

변수명은 보안을 위해 measurement_0과 같이 익명화되었다.

데이터 구성

학습데이터: train_prob.csv, 21,458 rows, 25 columns

테스트데이터: test_prob.csv, 5,112 rows, 24 columns, 

테스트정답셋: test_prob_ans.csv, 5,112 rows, 1 columns


컬럼명	설명	타입

|변수명|설명|타입|
|--|--------------|------|
|id|실험 고유 번호|정수형|
|product_code|스프링 코드|범주형|
|loading|스프링에 가한 부하|실수형|
|attribute_0|구성 소재1|범주형|
|attribute_1|구성 소재2|범주형|
|attribute_2|구성 소재3|정수형|
|attribute_3|구성 소재4|정수형|
|measurement_0 ~ 17|측정값 0~17|실수형|
|failure|성공여부|이진형(0, 1)|


# 전처리(Preprocessing)

train_prob.csv를 불러 온다. 이를 basetable이리고 한다.

In [13]:
df_basetable = pd.read_csv('train_prob.csv')

# 단계 1

basetable에 measurement_3 ~17 각각의 행이 결측인지 나타내는 파생 변수를 만든다. 

파생 변수는 이진 형식이고, False는 미결측 True는 결측을 의미한다. 

파생 변수의 이름은 measurement 번호에 따라 isna_3 ~ 17로 한다. 

In [14]:

for i in range(3, 18):
    df_basetable['isna_%d'%i] = df_basetable['measurement_%d'%i].isna()
    
df_basetable.head()

,id,product_code,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,...,isna_8,isna_9,isna_10,isna_11,isna_12,isna_13,isna_14,isna_15,isna_16,isna_17
0,0,A,80.10,material_7,material_8,9,5,7,8,4,...,False,False,False,False,False,False,True,False,False,False
1,1,A,84.89,material_7,material_8,9,5,14,3,3,...,False,False,False,False,False,False,False,False,False,False
2,2,A,82.43,material_7,material_8,9,5,12,1,5,...,False,False,False,True,False,False,False,False,False,False
3,3,A,101.07,material_7,material_8,9,5,13,2,6,...,False,False,False,False,False,False,False,False,False,False
4,4,A,188.06,material_7,material_8,9,5,9,2,8,...,False,False,False,False,False,False,False,False,False,False


## 단계 2

이 과제를 맡은 데이터분석가 지희는 measurement_3~17의 결측치 처리 방안을 고민하던 중, 

개발부서에서 measurement_17은 product_code별로 failure를 예측하기 위해 

measurement_3 ~ measurement_9을 다음과 같이 선형 조합하여 생성한 값이라는 정보를 받았다. 

$measurement_{17}= \beta_{0} + \beta_{3}measurement_{3}+\beta_{4}measurement_{4}+...+\beta_{9}measurement_{9}$

이는 즉,

$measurement_{3}= \beta'_{0} + \beta'_{4}measurement_{4}+\beta'_{5}measurement_{5}+...+\beta'_{9}measurement_{9} + \beta'_{17}measurement_{17}$

...

$measurement_{9}= \beta''_{0} + \beta''_{3}measurement_{3}+\beta''_{4}measurement_{4}+...+\beta''_{8}measurement_{8}+\beta''_{17}measurement_{17}$

와 같이 measurement_3 ~ measurement_9의 각 변수들도 나머지 변수들과 선형 관계를 지닌다. 

이 점을 이용하여 대상 변수를 번갈아 가면서 예측 모델을 만들어 최대한 원래 값에 가깝게 복원할 수 있다. 

이러한 반복적인 결측치 복원 방법을 사내 데이터분석 연구소에 문의 했더니 다음과 같은 가이드를 주었다. 

> sklearn 모듈에 아직은 실험 단계이지만, 비슷한 경우에 문제 없이 사용했던 사례가 있어 의견을 드립니다. 

> from sklearn.experimental import enable_iterative_imputer 구문을 사용하여 실험 단계인 모듈을 활성화하고, 

> sklearn.impute.IterativeImputer를 사용한다면 원하는 결과를 얻을 수 있습니다.

가이드의 내용을 참조하여 basetable의 measurement_3~9와 measurement_17 결측치를 복원하라.


입력 변수] measurement_3 ~ 9, measurement_17 (입력 변수 순서에 유의)

---
**함수가이드**

sklearn.experimental.enable_iterative_imputer

sklearn.impute.IterativeImputer, random_state=123

sklearn.linear_model.LinearRegression

문제 지시사항 외 Default 값 사용

---


In [47]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression

cols = ['measurement_%d' % i for i in range(3, 10)] + ['measurement_17']
imputer = IterativeImputer(
        estimator = LinearRegression(),
        random_state=123
)
# result of imputer.fit_transform(x) has type of numpy. So need to make a new dataframe for this.
df_basetable[cols] = df_basetable.groupby('product_code')[cols].apply(lambda x : pd.DataFrame(imputer.fit_transform(x), columns=x.columns, index=x.index))

## 단계 3

measurement_10~16까지의 결측치는 모두 product_code별 평균으로 대치한다.

In [48]:
cols = ['measurement_%d' % i for i in range(10, 17)]

df_basetable[cols] = df_basetable.groupby('product_code')[cols].apply(lambda x: x.fillna(x.mean()))
df_basetable.head()

,id,product_code,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,...,isna_8,isna_9,isna_10,isna_11,isna_12,isna_13,isna_14,isna_15,isna_16,isna_17
0,0,A,80.10,material_7,material_8,9,5,7,8,4,...,False,False,False,False,False,False,True,False,False,False
1,1,A,84.89,material_7,material_8,9,5,14,3,3,...,False,False,False,False,False,False,False,False,False,False
2,2,A,82.43,material_7,material_8,9,5,12,1,5,...,False,False,False,True,False,False,False,False,False,False
3,3,A,101.07,material_7,material_8,9,5,13,2,6,...,False,False,False,False,False,False,False,False,False,False
4,4,A,188.06,material_7,material_8,9,5,9,2,8,...,False,False,False,False,False,False,False,False,False,False


Hint] 전처리 단계에서 보간 결과를 확인해 보기 위한 각 변수의 평균과 표본표준편차.

| |3|4|5|6|7|8|9|10|11|12|13|14|15|16|17|
|-|-|-|-|-|-|-|-|--|--|--|--|--|--|--|--|
|mean|17.796|11.736|17.131|17.506|11.719|19.022|11.434|16.034|19.194|11.734|15.666|16.033|15.051|16.398|701.768|
|std|0.997|0.994|0.994|0.992|0.993|1.005|0.997|1.278|1.579|1.433|1.149|1.461|1.478|1.671|119.180|

열의 이름의 숫자는 measurement_ 번호, 값은 소수점 3째 자리까지 반올림

In [51]:
cols = ['measurement_%d' % i for i in range(3, 18)]
df_basetable[cols].agg(['mean', 'std']).rename(columns=lambda x: x.split('_')[1]).round(3)

,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
mean,17.795,11.736,17.130,17.506,11.719,19.022,11.432,16.034,19.194,11.734,15.666,16.033,15.051,16.398,701.742
std,0.997,0.995,0.998,0.993,0.995,1.004,0.993,1.278,1.579,1.433,1.149,1.461,1.478,1.671,118.767


# 문제1

(basetable을 사용) measurement_3~16까지 결측 여부가 failure에 영향이 있는지를 파악하고, 

failure를 분류하는 데 도움이 될 만한 것은 예측 모델의 입력 변수로 사용하고자 한다. 

이를 위해 전처리 과정에서 뽑아낸 isna_3~16을 활용한다.

n이 3부터 16까지, 즉 measurement_3~16까지 다음의 검정을 수행한다. 

$H_0: P(failure=True|measurement_{n}=Missing)=P(failure=True)$

$H_1: P(failure=True|measurement_{n}=Missing) \neq P(failure=True)$

모집단의 $P(failure=True) = 0.2114$


## 단계 1-1

우선, measurement_3으로 위 검정을 시행해보자.

$H_0: P(failure=True|isna_{3}=True)=0.2114$

$H_1: P(failure=True|isna_{3}=True) \neq 0.2114$

으로 바꿀 수 있다.

$P(failure=True|isna_{3}=True)$은 표본수가 충분하여 중심극한정리에 의해 정규분포를 따르는 것은 분석가 간에 이견이 없다고 한다. 

위 검정의 p-value를 구하여 보고 힌트에 주어진 p-value와 비교하여 검정 방법에 문제가 없음을 확인하라.

---

**함수 가이드**

 scipy.stats 에서 제공 기능 활용
 
 문제 지시사항 외 Default 값 사용
 
---

 Hint] p-value는 0.0037(소수점 다섯째 자리에서 반올림하여 넷째 자리까지 표시)


In [66]:
from statsmodels.stats.proportion import proportions_ztest

# P(faiilure=True|isna_3=True) = P(failure=True and isna_3=True) / P(isna_3=True)

p_true = 0.2114
proportions_ztest(
    len(df_basetable.query('isna_3 == True and failure == True'))
    len(df_basetable.query('isna_3 == True')),
    p_true,
    alternative='two-sided',
    prop_var=p_true, # 지금은 모비율을 아는 상황이므로 proportion variance 계싼 시 그대로 사용
)

(-2.905807189028007, 0.0036630709140241546)

## 단계 1-2

measuremenet_3을 포함하여 measurement_4 ~ 16까지 위 검정을 반복하고 

귀무가설을 기각할 수 있는 경우의 p-value의 합을 A라고 한다. (유의 수준은 5%로 한다.)

In [72]:
from statsmodels.stats.proportion import proportions_ztest

# P(faiilure=True|isna_3=True) = P(failure=True and isna_3=True) / P(isna_3=True)

p_true = 0.2114
pvalues = []
indices = []
for i in range(3, 17):
    numer = 'isna_%d == True and failure == True' % i
    denom = 'isna_%d == True' % i
    pvalue = proportions_ztest(
        len(df_basetable.query(numer)),
        len(df_basetable.query(denom)),
        p_true,
        alternative='two-sided',
        prop_var=p_true, # 지금은 모비율을 아는 상황이므로 proportion variance 계싼 시 그대로 사용
    )[1]
    if pvalue < 0.05:
        pvalues.append(pvalue)
        indices.append(i)
        
A = sum(pvalues)
pvalues, indices, sum(pvalues)

([0.0036630709140241546, 0.026325008335499278], [3, 5], 0.029988079249523434)

## 단계 1-3

검정 결과 귀무가설을 기각할 수 있는 경우는 총 두 건이다. 

해당 파생 변수명의 뒷 자리 번호 순으로 na_1, na_2로 파생 변수를 만들어 prob1 데이터셋을 생성하라.

In [73]:
df_prob1 = df_basetable.copy()
df_prob1['na_1'] = df_prob1['isna_3']
df_prob1['na_2'] = df_prob1['isna_5']
round(A, 3)

0.03

A의 값을 소수점 넷째 자리에서 반올림하여 셋째 자리까지 출력하시오. 


# 문제 2

첫째는 스프링 개발 업체들은 실험이 제품 별로 공정하게 진행이 됐는지를 의문을 가지고 있다.

product_code에 따라 개발 업체가 다르다. 

product_code에 대해서 스프링에 가한 부하(loading)를 동일하게 했는지 조사하라.

둘째는, attribute_0와 attribute_1은 스프링을 구성하는 주요 소재이다. 

failure와는 관계가 없음이 이전에 검증되었다. 

하지만, 이에 대한 재확인 요청을 받아 attribute_0와 attribute_1은 failure와 상관없음을 확인한다.

이를 위해 다음 단계를 수행하라.


## 단계 2-1

prob1에서 입력 변수 loading에 결측이 없는 행들을 뽑아 prob2 데이터프레임을 만든다.

Hint] prob2의 데이터 수는 21,257 이다.


In [74]:
df_prob2 = df_prob1[~df_prob1['loading'].isna()]
df_prob2

,id,product_code,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,...,isna_10,isna_11,isna_12,isna_13,isna_14,isna_15,isna_16,isna_17,na_1,na_2
0,0,A,80.10,material_7,material_8,9,5,7,8,4,...,False,False,False,False,True,False,False,False,False,False
1,1,A,84.89,material_7,material_8,9,5,14,3,3,...,False,False,False,False,False,False,False,False,False,False
2,2,A,82.43,material_7,material_8,9,5,12,1,5,...,False,True,False,False,False,False,False,False,False,False
3,3,A,101.07,material_7,material_8,9,5,13,2,6,...,False,False,False,False,False,False,False,False,False,False
4,4,A,188.06,material_7,material_8,9,5,9,2,8,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21453,26565,E,158.95,material_7,material_6,6,9,6,16,4,...,False,False,False,False,False,False,True,False,False,False
21454,26566,E,146.02,material_7,material_6,6,9,10,12,8,...,False,False,False,False,False,False,False,False,False,False
21455,26567,E,115.62,material_7,material_6,6,9,1,10,1,...,False,False,False,False,False,False,False,False,False,False
21456,26568,E,106.38,material_7,material_6,6,9,2,9,4,...,False,False,False,False,False,False,False,False,False,False


## 단계 2-2

prob2에 loading의 각 행들에 자연 로그 함수를 적용하여 파생 변수 loading_log를 만든다.


In [80]:
import math
df_prob2['loading_log'] = df_prob2['loading'].apply(lambda x: np.log(x))
df_prob2

C:\Users\xptr3\AppData\Local\Temp\ipykernel_36588\2065584175.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_prob2['loading_log'] = df_prob2['loading'].apply(lambda x: np.log(x))


,id,product_code,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,...,isna_11,isna_12,isna_13,isna_14,isna_15,isna_16,isna_17,na_1,na_2,loading_log
0,0,A,80.10,material_7,material_8,9,5,7,8,4,...,False,False,False,True,False,False,False,False,False,4.383276
1,1,A,84.89,material_7,material_8,9,5,14,3,3,...,False,False,False,False,False,False,False,False,False,4.441356
2,2,A,82.43,material_7,material_8,9,5,12,1,5,...,True,False,False,False,False,False,False,False,False,4.411949
3,3,A,101.07,material_7,material_8,9,5,13,2,6,...,False,False,False,False,False,False,False,False,False,4.615813
4,4,A,188.06,material_7,material_8,9,5,9,2,8,...,False,False,False,False,False,False,False,False,False,5.236761
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21453,26565,E,158.95,material_7,material_6,6,9,6,16,4,...,False,False,False,False,False,True,False,False,False,5.068590
21454,26566,E,146.02,material_7,material_6,6,9,10,12,8,...,False,False,False,False,False,False,False,False,False,4.983744
21455,26567,E,115.62,material_7,material_6,6,9,1,10,1,...,False,False,False,False,False,False,False,False,False,4.750309
21456,26568,E,106.38,material_7,material_6,6,9,2,9,4,...,False,False,False,False,False,False,False,False,False,4.667018


## 단계 2-3

loading_log가 product_code 각각에 대해서 정규성을 지니고 있는지 확인하고자 한다.

이를 위해 product_code 별로 loading_log에 대한 Jarque-Bera로 검정하고, 

검정 결과 정규성을 부정할 수 없는 경우의 product_code의 수를 B라고 한다.(유의 수준: 5%)

---

**함수 가이드**

 scipy.stats 에서 제공 기능 활용
 
 문제 지시사항 외 Default 값 사용
 
---


In [84]:
from scipy.stats import jarque_bera

B = df_prob2.groupby('product_code')['loading_log'].apply(lambda x: jarque_bera(x).pvalue > 0.05).sum()
B

3

## 단계 2-4

loading_log 변수를 product_code로 구분했을 때, 

등분산성을 보이는지 Bartlett 검정을 통해 확인한다.

검정 결과에서 p-value를 C라고 한다.

---

**함수 가이드**

 scipy.stats 에서 제공 기능 활용
 
 문제 지시사항 외 Default 값 사용
 
---


In [90]:
from scipy.stats import bartlett 

results = []
def push(x):
    results.append(x)
    
df_prob2.groupby('product_code')['loading_log'].apply(lambda x: push(x))

C = bartlett(
    results[0],
    results[1],
    results[2],
    results[3]
).pvalue
C

0.5873433093297675

## 단계 2-5

product_code에 대한 분산분석(ANOVA)을 통해서 loading_log 평균에 차이가 있는지 검정한다.

그 결과 중 p-value를 D라고 한다.

---

**함수 가이드**

 scipy.stats 제공 기능 활용
 
 문제 지시사항 외 Default 값 사용
 
---


In [91]:
from scipy.stats import f_oneway

D = f_oneway(*results).pvalue
D 

0.7733782072320899

## 단계 2-6

카이제곱검정을 통해 attribute_0, attribute_1의 결합값이 failure와 연관이 있는지 조사하라. 

attribute_0, attribute_1의 결합값의 의미 attribute_0=material_7, attribute_1=material_8 이라면, 이 둘의 결합값은
matertial_7material_8를 의미한다.

(유의 수준 1%) 연관이 있다면 E값은 1 없으면 0으로 한다.

---

**함수 가이드**

 scipy.stats.chi2_contingency, correction=False
 
 문제 지시사항 외 Default 값 사용
 
---


In [92]:
df_prob2.head()

,id,product_code,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,...,isna_11,isna_12,isna_13,isna_14,isna_15,isna_16,isna_17,na_1,na_2,loading_log
0,0,A,80.10,material_7,material_8,9,5,7,8,4,...,False,False,False,True,False,False,False,False,False,4.383276
1,1,A,84.89,material_7,material_8,9,5,14,3,3,...,False,False,False,False,False,False,False,False,False,4.441356
2,2,A,82.43,material_7,material_8,9,5,12,1,5,...,True,False,False,False,False,False,False,False,False,4.411949
3,3,A,101.07,material_7,material_8,9,5,13,2,6,...,False,False,False,False,False,False,False,False,False,4.615813
4,4,A,188.06,material_7,material_8,9,5,9,2,8,...,False,False,False,False,False,False,False,False,False,5.236761


In [98]:
from scipy.stats import chi2_contingency

s_attr = df_prob2['attribute_0'] + df_prob2['attribute_1']
df_conti = pd.crosstab(index=s_attr, columns=df_prob2['failure'])
display(df_conti)

result = chi2_contingency(df_conti, correction=False)
if result[1] > 0.01: # 두 변수는 서로 독립
    E = 0
else: # 두 변수는 연관이 있다
    E = 1

failure,0,1
row_0,,
material_5material_5,4162,1047
material_7material_6,4181,1100
material_7material_8,8413,2354


In [99]:
round(B + C + D + E, 2)

4.36

B + C + D + E의 값을 소수점 셋째 자리에서 반올림하여 둘째 자리까지 출력하시오.


# 문제 3

로지스틱 회귀모델로 수치형 변수 measurement_0 ~ 17, 

loading과 이진형인 na_1, na_2 중에서 최적의 성능을 보이는 입력 변수들을 찾고자 한다.


## 단계 3-1

prob1을 복사하여 prob3을 만든다. loading의 결측치는 loading의 평균으로 대치한다.  

In [156]:
df_prob3 = df_prob1.copy()
df_prob3 = df_prob3.fillna(df_prob3['loading'].mean())
df_prob3.head()

,id,product_code,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,...,isna_10,isna_11,isna_12,isna_13,isna_14,isna_15,isna_16,isna_17,na_1,na_2
0,0,A,80.10,material_7,material_8,9,5,7,8,4,...,False,False,False,False,True,False,False,False,False,False
1,1,A,84.89,material_7,material_8,9,5,14,3,3,...,False,False,False,False,False,False,False,False,False,False
2,2,A,82.43,material_7,material_8,9,5,12,1,5,...,False,True,False,False,False,False,False,False,False,False
3,3,A,101.07,material_7,material_8,9,5,13,2,6,...,False,False,False,False,False,False,False,False,False,False
4,4,A,188.06,material_7,material_8,9,5,9,2,8,...,False,False,False,False,False,False,False,False,False,False


## 단계 3-2: 
    
prob3를 80%는 학습데이터 prob3_train으로 20%는 테스트데이터 prob3_test로 나눈다. 

prob3_train의 failure가 1인 비율과 prob3_test의 failure가 1의 비율을 동일하게 한다.


---

**함수 가이드**

 sklearn.model_selection.train_test_split, random_state=123, 
 
 train과 test의 failure의 비율은 stratify 매개 변수를 이용하여 맞춘다.
 
 문제 지시사항 외 Default 값 사용
 
---

In [157]:
from sklearn.model_selection import train_test_split

df_prob3_train, df_prob3_test = train_test_split(df_prob3, random_state=123, train_size=0.8, test_size=0.2, stratify=df_prob3['failure'])
df_prob3_train = df_prob3_train.copy()
df_prob3_test = df_prob3_test.copy()

In [158]:
df_prob3_train['failure'].value_counts(normalize=True)[0], df_prob3_test['failure'].value_counts(normalize=True)[0]

(0.788535477105907, 0.7886766076421249)

## 단계 3-3

prob3_train의 수치형 입력 변수 loading, measurement_0 ~ 17을 표준화한다. 

prob3_train의 표준화 설정으로 prob3_test의 loading, measurement_0 ~ 17에도 적용한다. 

표준화 처리한 prob3_train과 prob3_test는 문제 4와 문제 5에서 사용한다.


---

**함수 가이드**

 sklearn.preprocessing 제공 기능 활용, 
 
 문제 지시사항 외 Default 값 사용
 
---


In [159]:
from sklearn.preprocessing import StandardScaler
cols = ['loading'] + ['measurement_%d'%i for i in range(18)]
scaler = StandardScaler()
df_prob3_train[cols] = scaler.fit_transform(df_prob3_train[cols])
df_prob3_test[cols] = scaler.transform(df_prob3_test[cols])

In [160]:
df_prob3_train.head()

,id,product_code,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,...,isna_10,isna_11,isna_12,isna_13,isna_14,isna_15,isna_16,isna_17,na_1,na_2
12186,12186,C,-0.577100,material_7,material_8,5,8,-1.014834,0.988654,0.608313,...,False,False,False,False,False,False,False,False,False,True
7532,7532,B,1.853148,material_5,material_5,8,8,-1.250324,-0.929650,0.912362,...,False,False,False,False,False,False,False,False,False,False
5469,5469,B,0.509210,material_5,material_5,8,8,-0.779343,0.748866,1.824512,...,False,False,False,False,False,False,False,False,False,False
5998,5998,B,0.082142,material_5,material_5,8,8,-1.485815,-0.210286,0.000213,...,False,False,False,False,False,False,False,False,False,False
16713,21825,E,-0.570930,material_7,material_6,6,9,-1.014834,0.988654,-0.607887,...,True,False,False,False,False,False,False,False,False,False


In [161]:
df_prob3_test.head()

,id,product_code,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,...,isna_10,isna_11,isna_12,isna_13,isna_14,isna_15,isna_16,isna_17,na_1,na_2
6626,6626,B,-0.646778,material_5,material_5,8,8,-0.779343,-0.210286,0.912362,...,False,False,False,False,False,False,False,False,False,False
6302,6302,B,-0.776364,material_5,material_5,8,8,-0.308363,0.509078,1.216412,...,False,False,False,False,False,False,False,False,False,False
2068,2068,A,1.220389,material_7,material_8,9,5,1.104580,-1.409226,-0.911937,...,False,False,False,False,False,False,False,True,False,False
4661,4661,A,0.213270,material_7,material_8,9,5,-0.072872,-1.169438,-0.607887,...,False,False,False,False,False,False,True,False,False,False
10303,10303,B,0.694332,material_5,material_5,8,8,-0.072872,-0.689862,0.304263,...,False,False,False,False,False,False,False,False,False,False


## 단계 3-4
    
로지스틱 회귀모델을 사용하여 loading, measurement_0~17과 na_1, na_2를 입력 변수로 하여 prob3_train을 학습한다. 

로지스틱 회귀모델을 prob3_test로 성능을 측정한 값을 A라고 한다.

입력 변수: loading, measurement_0~17, na_1, na_2

대상 변수: failure

성능 지표: AUC(area under of ROC curve)

---

**함수 가이드**

 sklearn.linear_model.LogisticRegression, solver='lbfgs', 문제 지시사항 외 Default 값 사용
 
 sklearn.metrics.roc_auc_score
 
---

In [162]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

cols = ['loading'] + ['measurement_%d'%i for i in range(18)] + ['na_1', 'na_2']
model = LogisticRegression(solver='lbfgs')
model.fit(X=df_prob3_train[cols], y=df_prob3_train['failure'])
y_proba = model.predict_proba(X=df_prob3_test[cols])[:, 1]
print(y_proba)

A = roc_auc_score(df_prob3_test['failure'], y_proba) # (y_true, y_score)
A

[0.1689782  0.16316111 0.30370132 ... 0.17758263 0.198606   0.23437376]


0.5789029687039422

## 단계 3-5

loading, measurement_0 ~ 17, na_1, na_2를 후보 입력 변수로 한다. 

전진 선택법을 사용하여 이 후보 입력 변수 중에서 최적의 성능을 보이는 입력 변수의 조합을 찾는다. 

전진 선택법의 선택 기준은 prob3_train을 대상으로 5겹 층화교차검증(5-Fold stratified cross validation)을 하고 

겹외(OOF, Out-Of Fold) 성능의 평균값으로 한다. 전진 선택 과정에서 선택했던 변수를 제외하지 않는다. 

입력 변수: 본 단계 요건 참고

대상 변수: failure 

성능 지표: AUC(area under of ROC curve)

---
**함수가이드**

mlxtend.feature_selection.SequentialFeatureSelector

sklearn.linear_model.LogisticRegression, solver='lbfgs'

sklearn.metrics.roc_auc_score

sklearn.model_selection.StratifiedKFold, random_state=123, shuffle=True

문제 지시사항 외 Default 값 사용

---

In [163]:
from mlxtend.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold

LogisticRegression(solver='lbfgs')


sfs = SequentialFeatureSelector( # variable selector
    estimator=LogisticRegression(solver='lbfgs'),
    forward=True, # 전진선택법
    k_features='best',
    cv=StratifiedKFold(random_state=123, shuffle=True), # 5겹 층화교차검증(5-fold stratified cv)
    floating=False, # 선택했던 변수는 제외하지 않는다.
    scoring='roc_auc'
)
cols = ['loading'] + ['measurement_%d'%i for i in range(18)] + ['na_1', 'na_2']
sfs.fit(df_prob3_train[cols], df_prob3_train['failure'])

X_sfs_best = list(sfs.k_feature_names_)
X_sfs_best

['loading',
 'measurement_1',
 'measurement_4',
 'measurement_9',
 'measurement_13',
 'measurement_14',
 'measurement_16',
 'measurement_17',
 'na_1']

## 단계 3-6

단계 3-5에서 찾은 최적의 입력 변수 조합으로 로지스틱 회귀모델을 사용하여 prob3_train을 학습하고 

prob3_test로 성능을 측정한 값을 B라고 한다.

입력 변수: **단계 3-5**에서 도출한 최적의 입력 변수 조합

대상 변수: failure

성능 지표: AUC(area under of ROC curve)

---
**함수 가이드**

sklearn.linear_model.LogisticRegression, solver='lbfgs'

sklearn.metrics.roc_auc_score

문제 지시사항 외 Default 값 사용

---

In [164]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

cols = ['loading',
 'measurement_1',
 'measurement_4',
 'measurement_9',
 'measurement_13',
 'measurement_14',
 'measurement_16',
 'measurement_17',
 'na_1']

model = LogisticRegression(solver='lbfgs')
model.fit(X=df_prob3_train[cols], y=df_prob3_train['failure'])
y_proba = model.predict_proba(X=df_prob3_test[cols])[:, 1]
print(y_proba)

B = roc_auc_score(df_prob3_test['failure'], y_proba) # (y_true, y_score)
B

[0.16794297 0.16487401 0.2939887  ... 0.17267039 0.1987921  0.22876432]


0.5815646888878394

In [165]:
round(A-B, 3)

-0.003

A-B값을 소수점 넷째 자리에서 반올림하여 셋째 자리까지 출력하시오


# 문제 4

차원 축소 기법을 통한 데이터의 특성과 failure 분류 성능을 높힐 만한 요소를 살펴 본다. 

첫째로, loading을 제외하고, measurement_0 ~ 17을 입력으로 failure를 대상 변수로 Linear Discrimant Analysis(LDA) 모델을 만든다. 

True/False 이진 변수인 failure를 분류한다는 점에서 LDA 모델은 measurement_0 ~ 17를 한 개의 경계점으로 

failure를 최대한 정확하게 구분하도록 하나의 연속형 변수로 변환한다. 

스프링의 내구력이 높을 수록 failure 확률이 낮아진다면, 

측정값 measurement_0 ~ 17의 LDA 변환값은 스프링의 내구력을 나타낸다라고 할 수 있다.

실험에서 스프링에 가한 부하(loading)와  LDA 변환값의 상관도를 측정하여, 

측정값 measurement_0~17 에서 예상되는 내구력이 스프링에 따라 부하(loading)의 반영 정도를 가늠한다.

둘째로, PCA를 사용하여 차원 감소로 failure 분류 성능에 얼마나 효과가 있을지 살펴본다.

문제3에서 사용했던, 전처리(loading 결측치 처리와 표준화 과정을 거친) 과정을 거친 prob3_train과 prob3_test를 사용한다.


## 단계 4-1

prob3_train에서 measurement_0 ~ 17을 입력으로 failure를 대상 변수로 하여 LDA(Linear Discriminant Analysis) 모델을 학습한다. 

measurement_0 ~ 17에 대한 prob3_train에서의 LDA의 변환값과 loading과 스피어만 상관도 (spearman correlation)의 p-value를 구하여 A라고 한다.

입력 변수] measurement_0 ~ 17 (순서에 유의 하시오)

대상 변수] failure

---
**함수가이드**

sklearn.discriminant_analysis 제공 기능 활용

scipy.stats.spearmanr

문제 지시사항 외 Default 값 사용

---

In [166]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from scipy.stats import spearmanr

cols = ['measurement_%d'%i for i in range(18)]
lda = LinearDiscriminantAnalysis()
result = lda.fit_transform(X=df_prob3_train[cols], y=df_prob3_train['failure'])

A = spearmanr(result, df_prob3_train['loading']).pvalue
A

0.6997463328801665

## 단계 4-2

prob3_train에서 measurement_0 ~ 17을 대상으로 주성분분석(Principal Component Analysis, PCA) 모델을 학습한다. 

분산 설명율이 높은 순으로 주성분을 변수명을 pca_0 ~ 17하여 prob3_train에 추가하여 prob4_train을 만든다. 

prob3_test에 prob3_train를 학습했던 PCA 모델로 동일한 방법으로 pca_0 ~17 파생 변수를 추가하여 prob4_test를 만든다.

입력 변수] measurement_0 ~ 17 (순서에 유의 하시오)

---
**함수가이드**

sklearn.decomposition.PCA, random_state=123

문제 지시사항 외 Default 값 사용

---


In [167]:
from sklearn.decomposition import PCA

pca = PCA(n_components=None, random_state=123) # n_components: 모든 성분 가져옴
cols = ['measurement_%d'%i for i in range(18)]
pca_cols = ['pca_%d'%i for i in range(18)]
df_prob4_train = df_prob3_train.copy()
df_prob4_test = df_prob3_test.copy()

df_prob4_train[pca_cols] = pd.DataFrame(pca.fit_transform(X=df_prob3_train[cols]), index=df_prob3_train.index, columns=pca_cols)
df_prob4_test[pca_cols] = pd.DataFrame(pca.transform(X=df_prob3_test[cols]), index=df_prob3_test.index, columns=pca_cols)

In [168]:
df_prob4_train.head()

,id,product_code,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,...,pca_8,pca_9,pca_10,pca_11,pca_12,pca_13,pca_14,pca_15,pca_16,pca_17
12186,12186,C,-0.577100,material_7,material_8,5,8,-1.014834,0.988654,0.608313,...,0.366010,-1.232174,1.428001,1.389962,-0.398195,-0.739423,0.121253,0.199190,0.157694,-0.464549
7532,7532,B,1.853148,material_5,material_5,8,8,-1.250324,-0.929650,0.912362,...,0.035812,-1.169482,0.489195,-1.002865,0.134384,-0.572542,1.524020,0.656114,-1.358946,-0.364510
5469,5469,B,0.509210,material_5,material_5,8,8,-0.779343,0.748866,1.824512,...,1.392416,-1.174218,-0.304001,1.011597,1.160155,0.152649,-1.410324,0.530532,0.640403,-1.138164
5998,5998,B,0.082142,material_5,material_5,8,8,-1.485815,-0.210286,0.000213,...,0.481474,-0.659595,-0.281410,2.103323,-0.345672,0.316510,0.279006,1.091862,-1.006757,-0.211065
16713,21825,E,-0.570930,material_7,material_6,6,9,-1.014834,0.988654,-0.607887,...,-1.396104,0.064487,2.872736,-0.210350,0.650109,1.019575,-3.054150,0.606982,-0.067877,-1.029491


## 단계 4-3

초기에 loading을 입력 변수로 하여 prob4_train을 학습하고, prob4_test에 대한 성능을 측정한다.

여기에 pca_0에서 pca_17까지 입력 변수를 하나씩 추가 하면서, 

즉 분산 설명율이 높은 순으로 컴포넌트를 하나씩 추가하여 prob4_train를 학습하고 prob4_test의 성능을 측정 했을 때, 

최적의 성능을 보인 컴포넌트들의 분산 설명율의 합을 B라고 한다. (만일 없다면 B = 0이다.)

입력 변수: 설명 참고

대상 변수: failure

성능 지표: AUC(area under of ROC curve)

---
**함수가이드**

sklearn.linear_model.LogisticRegression, solver=’lbfgs’

---

In [172]:
from sklearn.linear_model import LogisticRegression


cols = ['loading'] + ['pca_%d' % i for i in range(18)]
scores = []
for i in range(1, len(cols)+1):
    X_vars = cols[:i]
    model = LogisticRegression(solver='lbfgs')
    model.fit(X=df_prob4_train[X_vars], y=df_prob4_train['failure'])
    score = roc_auc_score(df_prob4_test['failure'], model.predict_proba(df_prob4_test[X_vars])[:, 1])
    scores.append(score)
    print(X_vars[-1], score)
    
np.argmax(scores)

loading 0.5778277926972065
pca_0 0.5793534286910115
pca_1 0.5782346072480739
pca_2 0.5785586908974837
pca_3 0.5800051788241464
pca_4 0.5804647587531085
pca_5 0.5809572356153274
pca_6 0.5813617701807214
pca_7 0.580008435946251
pca_8 0.5797032436050479
pca_9 0.5796495010903215
pca_10 0.5788332662909034
pca_11 0.5785684622637975
pca_12 0.5777532046010107
pca_13 0.577810529950052
pca_14 0.57729460180868
pca_15 0.576876713042657
pca_16 0.5764493786225304
pca_17 0.5767952849900414


7

In [175]:
B = pca.explained_variance_ratio_[:7].sum()
B

0.49017371228143375

In [176]:
round(A+B, 2)

1.28

In [178]:
pca.explained_variance_ # eigen value

array([1.84854205, 1.41906121, 1.25534603, 1.16331717, 1.06601416,
       1.05018379, 1.02117642, 1.01051939, 1.00814692, 0.99368603,
       0.98521424, 0.97709589, 0.97242708, 0.8755711 , 0.81405554,
       0.70774444, 0.67972429, 0.15322287])

In [179]:
pca.components_ # eigenvectors

array([[ 2.44015283e-02, -1.89141822e-02, -1.65225903e-02,
         5.12598345e-02,  2.03888440e-01,  3.71627917e-01,
         1.90370916e-01,  2.67725998e-01,  4.27321824e-01,
         1.58361752e-01, -9.05986096e-03, -2.20162543e-02,
         3.14980652e-03, -3.48331889e-02,  1.18410089e-02,
        -1.86813692e-02, -2.79463157e-02,  7.05554744e-01],
       [ 5.26351835e-01, -5.62333455e-01, -1.03542935e-01,
         1.64800115e-02, -4.24100353e-02,  3.57226416e-02,
         9.17172105e-04,  2.48574005e-02, -6.16169394e-02,
        -6.14491250e-05,  7.89894790e-03,  2.45580700e-01,
         3.80418220e-01, -1.44541991e-01, -1.13098702e-01,
        -3.83888979e-01,  3.93830433e-02, -2.35844954e-02],
       [-1.86021474e-01,  7.46638780e-02,  1.42650985e-01,
         4.32897697e-03,  5.31960355e-02,  7.75435603e-03,
         2.37037677e-02, -1.48914709e-02,  3.18229865e-02,
        -1.99467148e-02, -9.26139986e-02,  3.65079778e-01,
         2.86327308e-01,  2.98825246e-01, -3.96665489e

A + B를 소수점 셋째 자리에서 반올림하여 둘째 자리까지 구하라.


# 문제 5

랜덤포레스트 분류기(Random-Forest Classifier)의 최적의 하이퍼 파라미터(Hyper-Parameter, 초매개변수)를 탐색하고자 한다.

문제3에서 사용했던, 전처리(loading 결측치 처리와 표준화 과정을 거친) 과정을 거친 prob3_train과 prob3_test를 사용한다.

## 단계 5-1

sklearn에서 제공하는 랜덤포레스트 분류기(Random-Forest Classifier)의 하이퍼 파라미터 중 

n_estimators, max_depth 그리고 min_samples_split의 최적 조합을 탐색한다. 

탐색 값은 아래에 제공한 하이퍼 파라미터의 모든 조합이다. 

prob3_train을 대상으로 5-겹 층화교차검증(5-fold stratified cross validation)으로 

각각  겹외셋(OOF set, Out-Of-Fold set)의 성능에 대한 평균을 기준으로 하이퍼 파라미터를 선택한다.

  - n_estimators: [5, 10, 15]

  - max_depth: [5, 6, 7]
  
  - min_samples_split: [256, 512]

Hint] 모든 하이퍼 파라미터의 조합의 수는 18개이다

입력 변수: loading, measurement_0 ~ 17, na_1, na_2 (순서에 유의)

대상 변수: failure

성능 지표: AUC (area under of ROC curve)

---
**함수가이드**

sklearn.ensemble.RandomForestClassifier, random_state=123 

itertools.product 필요시 사용

sklearn.model_selection.cross_val_score 필요시 사용

sklearn.model_selection.StratifiedKFold, random_state=123, shuffle=True

sklearn.model_selection.GridSearchCV 필요시 사용

---

In [182]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

cols = ['loading'] + ['measurement_{}'.format(i) for i in range(0, 18)] + ['na_1', 'na_2']
gscv = GridSearchCV(
    estimator=RandomForestClassifier(random_state=123),
    param_grid={
        'n_estimators': [5, 10, 15],
        'max_depth': [5, 6, 7],
        'min_samples_split': [256, 512]
    },
    iid=False, # OOF set의 성능에 대한 평균
    cv=StratifiedKFold(5, random_state=123, shuffle=True), # 5-fold stratified cv
    scoring='roc_auc'
)
gscv.fit(X=df_prob3_train[cols], y=df_prob3_train['failure'])

TypeError: __init__() got an unexpected keyword argument 'iid'

## 단계 5-2

단계 5-1에서 구한 최적 하이퍼 파라미터로 설정한 랜덤포레스트 분류기(Random-Forest Classifier)를 사용하여 prob3_train 학습하고, 

prob3_test로 성능을 측정하여 이 값을 A라고 한다.

입력 변수: loading, measurement_0 ~ 17, na_1, na_2 (순서에 유의)

대상 변수: failure

성능 지표: AUC (area under of ROC curve)

---
**함수가이드**

sklearn.ensemble.RandomForestClassifier, random_state=123 

---


A값을 소수점 넷째 자리에서 반올림하여 3째 자리까지 출력하시오.
